# Predict notebook

## Import libraries & setup

In [4]:
import pandas as pd, numpy as np
import sys,os,shutil,gc,re,json,glob,math,time,random,warnings
from tqdm import tqdm
from ensemble_boxes import *

from pathlib import Path
from dotenv import load_dotenv, find_dotenv

In [5]:
env_path = '/home/jovyan/digilut/ENV'
# env_path = '/home/jovyan/digilut/.env'
# env_path = find_dotenv()
load_dotenv(env_path)

True

In [6]:
DIR_RAW_DATA = os.getenv('DIR_RAW_DATA')
DIR_PROCESSED_DATA = os.getenv('DIR_PROCESSED_DATA')
DIR_PREDICTIONS = os.getenv('DIR_PREDICTIONS')
DIR_MODELS = os.getenv('DIR_MODELS')

## Generate predictions

In [5]:
!pwd

/home/jovyan/digilut/notebooks


In [9]:
!python test/predict.py 512 #>>/dev/null


Loads checkpoint by local backend from path: /home/jovyan/digilut/models/512/model_512_f0/epoch_12.pth
generating predictions for  10 slides
100%|███████████████████████████████████████| 2785/2785 [21:38<00:00,  2.15it/s]
saving predictions to /home/jovyan/digilut/data/predictions/512/test-512_0.pkl
Loads checkpoint by local backend from path: /home/jovyan/digilut/models/512/model_512_f1/epoch_12.pth
generating predictions for  10 slides
100%|███████████████████████████████████████| 2785/2785 [21:06<00:00,  2.20it/s]
saving predictions to /home/jovyan/digilut/data/predictions/512/test-512_1.pkl
Loads checkpoint by local backend from path: /home/jovyan/digilut/models/512/model_512_f2/epoch_12.pth
generating predictions for  10 slides
100%|███████████████████████████████████████| 2785/2785 [20:56<00:00,  2.22it/s]
saving predictions to /home/jovyan/digilut/data/predictions/512/test-512_2.pkl
Loads checkpoint by local backend from path: /home/jovyan/digilut/models/512/model_512_f3/epoch_1

In [ ]:
!python test/predict.py 640


In [ ]:
!python test/predict.py 768 


In [ ]:
!python test/predict.py 896


In [ ]:
!python test/predict.py 1024


## Save final predictions

In [7]:
def bbox_iou(box1, box2, xywh=True, GIoU=False, DIoU=False, CIoU=False, eps=1e-7):
    """
    Calculate Intersection over Union (IoU) of box1(1, 4) to box2(n, 4).

    Args:
        box1 (torch.Tensor): A tensor representing a single bounding box with shape (1, 4).
        box2 (torch.Tensor): A tensor representing n bounding boxes with shape (n, 4).
        xywh (bool, optional): If True, input boxes are in (x, y, w, h) format. If False, input boxes are in
                               (x1, y1, x2, y2) format. Defaults to True.
        GIoU (bool, optional): If True, calculate Generalized IoU. Defaults to False.
        DIoU (bool, optional): If True, calculate Distance IoU. Defaults to False.
        CIoU (bool, optional): If True, calculate Complete IoU. Defaults to False.
        eps (float, optional): A small value to avoid division by zero. Defaults to 1e-7.

    Returns:
        (torch.Tensor): IoU, GIoU, DIoU, or CIoU values depending on the specified flags.
    """

    # Get the coordinates of bounding boxes
    if xywh:  # transform from xywh to xyxy
        (x1, y1, w1, h1), (x2, y2, w2, h2) = box1.chunk(4, -1), box2.chunk(4, -1)
        w1_, h1_, w2_, h2_ = w1 / 2, h1 / 2, w2 / 2, h2 / 2
        b1_x1, b1_x2, b1_y1, b1_y2 = x1 - w1_, x1 + w1_, y1 - h1_, y1 + h1_
        b2_x1, b2_x2, b2_y1, b2_y2 = x2 - w2_, x2 + w2_, y2 - h2_, y2 + h2_

    else:  # x1, y1, x2, y2 = box1
        b1_x1, b1_y1, b1_x2, b1_y2 = box1.chunk(4, -1)
        b2_x1, b2_y1, b2_x2, b2_y2 = box2.chunk(4, -1)
        w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1 + eps
        w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1 + eps


    # Intersection area
    inter = (b1_x2.minimum(b2_x2) - b1_x1.maximum(b2_x1)).clamp_(0) * (
        b1_y2.minimum(b2_y2) - b1_y1.maximum(b2_y1)
    ).clamp_(0)

    # Union Area
    union = w1 * h1 + w2 * h2 - inter + eps

    # IoU
    iou = inter / union
    if CIoU or DIoU or GIoU:
        cw = b1_x2.maximum(b2_x2) - b1_x1.minimum(b2_x1)  # convex (smallest enclosing box) width
        ch = b1_y2.maximum(b2_y2) - b1_y1.minimum(b2_y1)  # convex height
        if CIoU or DIoU:  # Distance or Complete IoU https://arxiv.org/abs/1911.08287v1
            c2 = cw.pow(2) + ch.pow(2) + eps  # convex diagonal squared
            rho2 = (
                (b2_x1 + b2_x2 - b1_x1 - b1_x2).pow(2) + (b2_y1 + b2_y2 - b1_y1 - b1_y2).pow(2)
            ) / 4  # center dist**2
            if CIoU:  # https://github.com/Zzh-tju/DIoU-SSD-pytorch/blob/master/utils/box/box_utils.py#L47
                v = (4 / math.pi**2) * ((w2 / h2).atan() - (w1 / h1).atan()).pow(2)
                with torch.no_grad():
                    alpha = v / (v - iou + (1 + eps))
                return iou - (rho2 / c2 + v * alpha)  # CIoU
            return iou - rho2 / c2  # DIoU
        c_area = cw * ch + eps  # convex area
        return iou - (c_area - union) / c_area  # GIoU https://arxiv.org/pdf/1902.09630.pdf
    return iou  # IoU

from copy import deepcopy
def get_score(gts,d,T=0.5):
    d_tp = d[d.giou>=T].drop_duplicates(subset='gt',keep='first') # correctly matched/ uniquely paired to a gt and >= threshold
    tp = len(d_tp)
    d_fp = d[(~d['pr'].isin(d_tp['pr'].values) & (d.giou>=T))] # extra matches to a gt and >= threshold
    fp = len(d_fp)
    n_gt = len(gts)
    fn = n_gt - tp #unmatched gt
    eps = 0.1
    precision = tp/(tp+fp+eps)
    recall = tp/(tp+fn+eps)
    sc = ((5 * precision) * recall) /  ((4*precision) + recall+eps)
    return sc

def get_multi_score(gts,d,Ts):
    scs = []
    for T in Ts:
        sc = get_score(gts,d,T)
        scs.append(sc)
    sc = np.mean(scs)
    return sc

def calc_scores(GTS,PREDS,Ts):
    scores = []
    for slide_id in GTS.keys():
        gts = deepcopy(GTS[slide_id])
        gts = torch.from_numpy(np.array(gts))
        if not slide_id in PREDS:
            scores.append(0)
            continue

        prs = deepcopy(PREDS[slide_id])

        res = []
        for pr in prs:
            pr = torch.tensor(pr).unsqueeze(0)
            gious = bbox_iou(pr, gts, xywh=False, GIoU=True).flatten().tolist()
            # gious = bbox_iou(pr, gts, xywh=True, GIoU=True).flatten().tolist()
            pr = pr[0].tolist()
            r = [(str(gts[i].tolist()),str(pr),gious[i]) for i in range(len(gts))]
            res += r


        d = pd.DataFrame(res,columns=['gt','pr','giou']).sort_values(by='giou',ascending=False)
        sc = get_multi_score(gts,d,Ts)

        scores.append(sc)
    return np.mean(scores)



def get_norm_bbs(df_res_wsi):
  
  bb_norm = []
  bbs = []
  hs = []; ws = []
  for _,row in df_res_wsi.iterrows():
    x = deepcopy(row.bbox)
    ws.append(x[2]-x[0])
    hs.append(x[3]-x[1])

    bbs.append(deepcopy(x))
    x = [x[0]/row.swidth,x[1]/row.sheight,x[2]/row.swidth,x[3]/row.sheight]
    x = np.array(x).clip(0,1)
    assert (x<=1).all()
    bb_norm.append(x)

  df_res_wsi['bbox_norm'] = bb_norm
  df_res_wsi['bbox'] = bbs
  df_res_wsi['bbh'] = hs
  df_res_wsi['bbw'] = ws

  print(len(df_res_wsi),df_res_wsi.slide_id.nunique())
  display(df_res_wsi.describe())
  return df_res_wsi


def load_res(path):
  df_res = pd.read_pickle(path).rename(columns={'bbox':'bbox_raw'}).dropna()

  d1 = df_res.file_id.str.split('.').str[0].str.rsplit('_',n=3,expand=True).drop(columns=1).copy()
  d1.columns = ['slide_id','y','x']
  df_res['x'] = d1.x.astype(int)
  df_res['y'] = d1.y.astype(int)

  bbs = []
  for _,row in df_res.iterrows():
    bb = deepcopy(np.array(row.bbox_raw))
    x1,y1,x2,y2 = bb

    tile_00_x = row.x
    tile_00_y = row.y
    x1 = x1+tile_00_x; x2 = x2+tile_00_x
    y1 = y1+tile_00_y; y2 = y2+tile_00_y

    bb = np.array([x1,y1,x2,y2]).round().astype(int).tolist()
    assert np.min(bb)>=0
    bbs.append(bb)

  df_res['bbox'] = bbs
  df_res = get_norm_bbs(df_res)
  return df_res


def prune_results(df_val_wsi,df_res_wsi_wbf, nmul=1):
  df_res_wsi_pruned = []
  dd = df_res_wsi_wbf.sort_values(by=['conf'],ascending=False).copy()

  for slide_id,d in dd.groupby('slide_id'):
    n = df_val_wsi[df_val_wsi.slide_id==slide_id].shape[0]
    d = d.head(n*nmul)
    df_res_wsi_pruned.append(d)
  df_res_wsi_pruned = pd.concat(df_res_wsi_pruned)
  return df_res_wsi_pruned


def get_wbf_multi_model(dfs,iou_thr,skip_box_thr,weights=None):
  wb_boxes = []; wb_scores = []; wb_labels = []
  df_res_wsi_wbf = []
  slide_ids = pd.concat(dfs).slide_id.unique().tolist()
  df_slide_meta = pd.concat(dfs)[['slide_id','swidth','sheight']].drop_duplicates()

  for slide_id in slide_ids:
    boxes_list = []; scores_list = []; labels_list = []
    for ix_res, df_res in enumerate(dfs):
      d = df_res[df_res.slide_id==slide_id].copy()
      # d = d[d['Tissue %']>1]
      d = d[d['Score']>0.01] #TODO
      # d = d[d.conf>0.8]
      boxes = d.bbox_norm.tolist()
      #assert len(boxes)>0
      scores = d.conf.tolist()
      labels = [0] * len(boxes)

      boxes_list.append(boxes)
      scores_list.append(scores)
      labels_list.append(labels)

    if weights is None:
      boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=[1 for i in range(len(boxes_list))], iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    else:
      boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=[weights[i] for i in range(len(boxes_list))], iou_thr=iou_thr, skip_box_thr=skip_box_thr)
   
    h = df_slide_meta[df_slide_meta.slide_id==slide_id].sheight.values[0]
    w = df_slide_meta[df_slide_meta.slide_id==slide_id].swidth.values[0]
    # h = d.sheight.values[0] #slide height
    # w = d.swidth.values[0] #slide width
    boxes = [[x[0]*w,x[1]*h,x[2]*w,x[3]*h] for x in boxes]
    ws = [b[2]-b[0] for b in boxes]
    hs = [b[3]-b[1] for b in boxes]
    d_res = pd.DataFrame(dict(slide_id=slide_id,conf=scores,bbox=boxes,bbh1=hs,bbw1=ws))
    df_res_wsi_wbf.append(d_res)


  df_res_wsi_wbf = pd.concat(df_res_wsi_wbf)
  return df_res_wsi_wbf



In [8]:
df_test = pd.read_csv(f'{DIR_RAW_DATA}/validation.csv')
df_test['slide_id'] = df_test.filename.str.split('.').str[0]
df_test = df_test.rename(columns={'max_x':'width','max_y':'height'})

In [16]:
PREDS = {}
for sz in [512,640,768,896,1024]:
  paths = glob.glob(f'{DIR_PREDICTIONS}/{sz}/*')
  paths = sorted(paths)
  preds = []
  for p in paths:
    res = load_res(p)
    preds.append(res)
  assert len(preds)==5
  PREDS[sz] = preds

In [11]:
dfs = PREDS[512]+PREDS[640]+PREDS[768]+PREDS[896]+PREDS[1024]
res_all = pd.concat(dfs)

weights = None
wts = [0.18062,0.21029,0.16048,0.21777,0.18230]
wts = [[wts[i]]*5 for i in range(5)]
# wts = np.array(wts)
wts = np.concatenate(wts)
weights = wts/wts.sum()

iou_thr,skip_box_thr =  0., 0. 

df_res_wsi_wbf = get_wbf_multi_model(dfs,iou_thr=iou_thr, skip_box_thr=skip_box_thr,weights=weights)
df_res_wsi_pruned =  prune_results(df_test,df_res_wsi_wbf, nmul=1)
len(df_res_wsi_pruned),df_res_wsi_pruned.slide_id.nunique(),res_all.shape

(119, 10, (3217, 32))

In [13]:
df_sub = df_test.copy()
df_sub['x1'] = None
df_sub['x2'] = None
df_sub['y1'] = None
df_sub['y2'] = None
sub = []

PAGE = 3
MUL = 2**PAGE
test_slides = df_sub.slide_id.unique()
for slide_id in sorted(test_slides):
  d = df_res_wsi_pruned[df_res_wsi_pruned.slide_id==slide_id].reset_index(drop=True)
  d1 = df_sub[df_sub.slide_id==slide_id].reset_index(drop=True)
  for ix,row in d.iterrows():
      x =  np.array(row.bbox)*MUL
      x1,y1,x2,y2 = x.round().astype(int)
      d1.loc[ix,'x1'] = x1
      d1.loc[ix,'y1'] = y1
      d1.loc[ix,'x2'] = x2
      d1.loc[ix,'y2'] = y2

  sub.append(d1)
df_sub = pd.concat(sub).sort_values(by='trustii_id').reset_index(drop=True)#.drop(columns=['slide_id'])
df_sub = df_sub.rename(columns={'width':'max_x','height':'max_y'})

df_na = df_sub[df_sub.x1.isna()].copy()
if len(df_na)>0:
  sub = []
  df_sub = df_sub[~df_sub.trustii_id.isin(df_na.trustii_id.tolist())]
  for slide_id in df_na.slide_id.unique():
    d1 = df_na[df_na.slide_id==slide_id].reset_index(drop=True)
    d = res_all[res_all.slide_id==slide_id].copy().sort_values(by='conf',ascending=False).reset_index(drop=True).head(len(d1))
    print(len(d),len(d1))
    for ix,row in d.iterrows():
        x =  np.array(row.bbox)*MUL
        x1,y1,x2,y2 = x.round().astype(int)
        d1.loc[ix,'x1'] = x1
        d1.loc[ix,'y1'] = y1
        d1.loc[ix,'x2'] = x2
        d1.loc[ix,'y2'] = y2

    sub.append(d1)

  df_sub1 = pd.concat(sub).sort_values(by='trustii_id').reset_index(drop=True) 
  df_sub = pd.concat([df_sub,df_sub1])

df_sub = df_sub.sort_values(by='trustii_id').reset_index(drop=True).drop(columns=['slide_id'])
df_sub

In [14]:
cols = ['x1','y1','x2','y2']
df_sub.loc[df_sub.x1.isnull(), cols] = df_sub.groupby('filename')[cols].transform('mean')
try:
  df_sub[cols] = df_sub[cols].astype(int)
except:
  print('filling with 0')
  df_sub[cols] = df_sub[cols].fillna(0).astype(int)

assert df_sub.dropna().shape[0]==762

filling with 0


In [15]:
df_sub.to_csv(f'{DIR_PREDICTIONS}/predictions.csv',index=False)
df_sub

,filename,max_x,max_y,trustii_id,x1,x2,y1,y2
0,hqi5y2OzZy_b.tif,82944,198144,1,16821,17680,53248,54282
1,1xebGQuAM7_b.tif,82944,194048,2,51500,52769,55166,55935
2,8xGdkL0vZt_a.tif,82944,197632,3,63048,63793,79246,80330
3,LQj5lC48hB_a.tif,82688,196352,4,59722,60609,33418,34185
4,9NlPhYAFUH_a.tif,82688,197888,5,21507,22247,29370,30151
...,...,...,...,...,...,...,...,...
757,7kiGhyiFBZ_b.tif,82688,198144,758,0,0,0,0
758,M62FqXX2cW_a.tif,82688,198144,759,56391,57224,82016,82687
759,AIg925SQy8_a.tif,82688,198144,760,0,0,0,0
760,OXCUbLOBGD_a.tif,82688,197632,761,0,0,0,0


In [ ]:
# submission_schema.validate(predictions_df)